**Diplomado en Inteligencia Artificial y Aprendizaje Profundo**

# Modelo Logístico de Clasificación  con Tensorflow 2.X

##  Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Oleg Jarma, ojarmam@unal.edu.co
4. Maria del Pilar Montenegro, pmontenegro88@gmail.com

## Contenido

* [Introducción](#Introducción)
* [El modelo lineal de clasificación](#El-modelo-lineal-de-clasificación)
* [Importar los módulos requeridos](#Importar-los-módulos-requeridos)
* [Carga del conjunto de datos Iris](#Carga-del-conjunto-de-datos-Iris)
* [Acercamiento descriptivo a los datos](#Acercamiento-descriptivo-a-los-datos)
* [Separa features y targets](#Separa-features-y-targets)
* [Divide los datos: entrenamiento y validación](#Divide-los-datos:-entrenamiento-y-validación)
* [Normaliza los datos](#Normaliza-los-datos)
* [Construye la tuberia (pipeline) para la alimentación de datos de Tensorflow](#Construye-la-tuberia-(pipeline)-para-la-alimentación-de-datos-de-Tensorflow)
* [Entrenamiento del Modelo](#Entrenamiento-del-Modelo)
* [Predicciones](#Predicciones)
* [Validación](#Validación)



## Introducción

Este código fue tomado y  adaptado de [Google Colab](https://colab.research.google.com/drive/1qNxKmi0QpkunqTDdpXfVLlneG-NFDN9c). En este ejercicio usaremos el famoso conjunto de datos *iris*. Sin embargo no se usaran todos los datos, porque en este ejercicio vamos a introducir el modelo logístico clasico que permite separar en dos clases. Los datos de la primera clase son omitidos y los datos se recodifican para tener solamente dos clases. Próximamente usaremos todos los datos.

## El modelo lineal de clasificación

En este  modelo se tienen varias variables regresoras o explicativas de entrada y una variable dicotómica de salida.

El propósito central es construir un modelo para predecir la probabilidad de que los elementos del espacio de entrada pertenezcan a una de dos clases, las cuales denotaremos como 0 y 1 respectivamente.

Supongamos que tenemos dos variables $X_1$ y $X_2$ que se espera permitan predecir si un elemento del conjunto de entrada pertenence a una clase: clase 1 ($Y=1$) o clase 0 ($Y=0$).

El modelo desde el punto de vista estadístico se escribe como

$$
[Y_i|X_1=x_{i1},X_2=x_{i2}] \sim \text{Bernoulli}(\pi_i),
$$

en donde 

$$
\pi_i = \frac{1}{1 + exp(-[b +w_1x_{i1} + w_2x_{i2})]}, i =i,\cdots,N
$$

En el entrenamiento se encontraran los pesos $w_1,w_2,$ y el intercepto $b$ que minimizan una determinada función de pérdida, a partir de un conjunto de datos de entrenamiento. 


Una vez garantizado que la máquina generaliza bien, probando con los datos de validación, la expresión anterior se utiliza para predecir la probabilidad que un nuevo valor no observado en el espacio de entrada, digamos $(x_1,x_2)$ pertenezca a a una clase. 

Por construcción $\pi$ es la probabilidad que el elemento $x$ pertenezca a la clase 1. Por lo tanto si por ejemplo $\pi = 0.8$ para un elemento, entonces lo clasificamos en la clase 1. 


La idea central que está detrás de este tipo de modelos se puede apreciar en la siguete imagen.




<figure>
<center>
<img src="../Imagenes/clasificador_lineal.png" width="600" height="500" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Clasificador Lineal</p>
</figcaption>
</figure>


Se trata de un clasificador lineal simple. Vamos a suponer que la máquina de aprendizaje ya está entrenada, por lo que los parámetros $w,b$ están fijos.

Observe que la línea roja divide el espacio $\mathcal{R}^2$ en  tres regiones. La primera es justamente la recta, que corresponde a un modelo de regresión como se estuio en la lección de [regresión lineal](am_intro_regresion.ipynb). Sobre la línea se cumple la ecuación 

$$
wx+b =0.
$$

Por otro lado se tiene que si $wx+b=0$, entonces la probabilidad $\pi$ es dada en este caso por

$$
\pi = \frac{1}{1+exp(-(wx+b))} = \frac{1}{2}.
$$

La segunda región está a la derecha. Usted puede verificar que en este caso, para todos los valores de $x$ se tiene que  $wx+b>0$. Como consecuencia, se tiene que $\pi>\tfrac{1}{2}$. en el caso extremo para valores $x$ muy alejados hacia la derecha, se tiene que $wx+b\to \infty$ y en consecuencia $\pi\to 1$.


En la tercera región (a la izquierda) ocurre el comportamiento simétrico pero en el otro sentido. Ahora $wx+b<0$, para todos los valores de $x$.  Se tiene que $\pi<\tfrac{1}{2}$. En el caso extremo para valores $x$ muy alejados hacia la izquierda, se tiene que $wx+b\to -\infty$ y en consecuencia $\pi\to 0$.



### Conclusión

El separador lineal funciona de la siguiente forma en este caso.

1. Si $\pi(x)$ es mayor que 0.5, la clase que debe asigna es 1. Entre mayor es $\pi(x)$ mayor tranquilidad para asignar la clase 1 al elemento $x$ en el espacio se entrada.
2. Si $\pi(x)$ es menor que 0.5, la clase que debe asigna es 0. Entre mayor es $\pi(x)$ mayor tranquilidad para asignar la clase 0 al elemento $x$ en el espacio se entrada.
3. Si $\pi(x)=0.5$, no se puede asignar una clase. Para valores muy cercanos a 0.5, no se debe asignar una clase directamente. Si fuera necesario tomar una decisión, lo mejor es seleccionar la clase de forma aleatoria. Como regla de combate, si $0.48 \le \pi(x)\le 0.52$, seleccionar aleatoriamente.

## Importar los módulos requeridos

In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd
import seaborn as sb
import tensorflow as tf
from tensorflow import keras
from tensorflow.estimator import LinearClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

print(tf.__version__)

## Carga del conjunto de datos Iris


In [ ]:
# nombres de las columnas de los datos
col_names = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
target_dimensions = ['Setosa', 'Versicolor', 'Virginica']

# lee los datos
training_data_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_data_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")
training = pd.read_csv(training_data_path, names=col_names, header=0)
test = pd.read_csv(test_data_path, names=col_names, header=0)

In [ ]:
test

In [ ]:
# esta sección es para omitir la clase 0: "Setosa" y recodificar loa datos  de entrenamiento
training = training[training['Species'] >= 1]
training['Species'] = training['Species'].replace([1,2], [0,1])

# esta sección es para omitir la clase 0: "Setosa" y recodificar los datos  de validación
test = test[test['Species'] >= 1]
test['Species'] = test['Species'].replace([1,2], [0,1])

# omite los índices de los dos dataframes para poderlos concadenar
training.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

# concadena los dataframes
iris_dataset = pd.concat([training, test], axis=0)

In [ ]:
iris_dataset

In [ ]:
iris_dataset.index

## Acercamiento descriptivo a los datos

In [ ]:
iris_dataset.describe().transpose()

In [ ]:
sb.pairplot(iris_dataset, diag_kind="kde")

In [ ]:
correlation_data = iris_dataset.corr()
correlation_data.style.background_gradient(cmap='coolwarm', axis=None)

## Separa features y targets

In [ ]:
X_data = iris_dataset[[m for m in iris_dataset.columns if m not in ['Species']]]
Y_data = iris_dataset[['Species']]

In [ ]:
X_data

## Divide los datos: entrenamiento y validación

In [ ]:
training_features , test_features ,training_labels, test_labels = train_test_split(X_data , Y_data , test_size=0.2)

In [ ]:
print('No. of rows in Training Features: ', training_features.shape[0])
print('No. of rows in Test Features: ', test_features.shape[0])
print('No. of columns in Training Features: ', training_features.shape[1])
print('No. of columns in Test Features: ', test_features.shape[1])

print('No. of rows in Training Label: ', training_labels.shape[0])
print('No. of rows in Test Label: ', test_labels.shape[0])
print('No. of columns in Training Label: ', training_labels.shape[1])
print('No. of columns in Test Label: ', test_labels.shape[1])

In [ ]:
stats = training_features.describe()
stats = stats.transpose()
stats

In [ ]:
stats = test_features.describe()
stats = stats.transpose()
stats

## Normaliza los datos

In [ ]:
def norm(x):
  stats = x.describe()
  stats = stats.transpose()
  return (x - stats['mean']) / stats['std']

normed_train_features = norm(training_features)
normed_test_features = norm(test_features)

## Construye la tuberia (pipeline) para la alimentación de datos de Tensorflow

In [ ]:
def feed_input(features_dataframe, target_dataframe, num_of_epochs=10, shuffle=True, batch_size=32):
  def input_feed_function():
    dataset = tf.data.Dataset.from_tensor_slices((dict(features_dataframe), target_dataframe))
    if shuffle:
      dataset = dataset.shuffle(2000)
    dataset = dataset.batch(batch_size).repeat(num_of_epochs)
    return dataset
  return input_feed_function

train_feed_input = feed_input(normed_train_features, training_labels)
train_feed_input_testing = feed_input(normed_train_features, training_labels, num_of_epochs=1, shuffle=False)
test_feed_input = feed_input(normed_test_features, test_labels, num_of_epochs=1, shuffle=False)

## Entrenamiento del Modelo

In [ ]:
feature_columns_numeric = [tf.feature_column.numeric_column(m) for m in training_features.columns]

In [ ]:
logistic_model = LinearClassifier(feature_columns=feature_columns_numeric)

In [ ]:
feature_columns_numeric

In [ ]:
logistic_model.train(train_feed_input)

## Predicciones

In [ ]:
train_predictions = logistic_model.predict(train_feed_input_testing)
test_predictions = logistic_model.predict(test_feed_input)

In [ ]:
train_predictions_series = pd.Series([p['classes'][0].decode("utf-8")   for p in train_predictions])
test_predictions_series = pd.Series([p['classes'][0].decode("utf-8")   for p in test_predictions])

In [ ]:
train_predictions_df = pd.DataFrame(train_predictions_series, columns=['predictions'])
test_predictions_df = pd.DataFrame(test_predictions_series, columns=['predictions'])

In [ ]:
training_labels.reset_index(drop=True, inplace=True)
train_predictions_df.reset_index(drop=True, inplace=True)

test_labels.reset_index(drop=True, inplace=True)
test_predictions_df.reset_index(drop=True, inplace=True)

In [ ]:
train_labels_with_predictions_df = pd.concat([training_labels, train_predictions_df], axis=1)
test_labels_with_predictions_df = pd.concat([test_labels, test_predictions_df], axis=1)

## Validación

In [ ]:
def calculate_binary_class_scores(y_true, y_pred):
  accuracy = accuracy_score(y_true, y_pred.astype('int64'))
  precision = precision_score(y_true, y_pred.astype('int64'))
  recall = recall_score(y_true, y_pred.astype('int64'))
  return accuracy, precision, recall

- **accuracy_score**: En la clasificación con múltiples etiquetas, esta función calcula la precisión del subconjunto: el conjunto de etiquetas predichas para una muestra que coincide exactamente con el conjunto de etiquetas correspondiente en y_true.
- **precision_score**: es la razón $\frac{tp }{tp + fp}$ en donde $tp$ es el número de positivos verdadero y $fp$ el número de falsos positivos. El mejor valor es 1 y el peor valor es 0.
- **recall_score**:  es la relación $\frac{tp }{tp + fn}$ donde $tp$ es el número de verdaderos positivos y $fn$ el número de falsos negativos. El recuerdo es intuitivamente la capacidad del clasificador para encontrar todas las muestras positivas. El mejor valor es 1 y el peor valor es 0.

In [ ]:
train_accuracy_score, train_precision_score, train_recall_score = calculate_binary_class_scores(training_labels, train_predictions_series)
test_accuracy_score, test_precision_score, test_recall_score = calculate_binary_class_scores(test_labels, test_predictions_series)

print('Training Data Accuracy (%) = ', round(train_accuracy_score*100,2))
print('Training Data Precision (%) = ', round(train_precision_score*100,2))
print('Training Data Recall (%) = ', round(train_recall_score*100,2))
print('-'*50)
print('Test Data Accuracy (%) = ', round(test_accuracy_score*100,2))
print('Test Data Precision (%) = ', round(test_precision_score*100,2))
print('Test Data Recall (%) = ', round(test_recall_score*100,2))


In [ ]:
train_predictions_series

In [ ]:
train_predictions_df 

-[Regresar al inicio](#Contenido)